In [1]:
import pandas as pd
import numpy as np
import os

from pathlib import Path
import gdt

from Bio import Entrez

In [ ]:
nth_iteration = 1

In [3]:
# Defines all the global variables used in the script.
# Change these variables to match your local setup.
# The most_recent_gdt_file variable should be set to the path of the most recent GDT file,
# OR the stripped GDT file used in filter command, if applicable.

DATA_DIR = "../test/Test_group16"
AN_missing_gene_dict = "../test/Test_group16/AN_missing_gene_dict"
#most_recent_gdt_file = "../test/Test_group16/Test_group16.gdt"
remove_orfs = True
organelle_type = "MT"
gff_suffix = ".gff3"

Entrez.email = 'dupin@alunos.utfpr.edu.br'
Entrez.api_key = 'b3abc1ac7ae9ac035af84ec1abf895878d09'

In [4]:
most_recent_gdt_file = ""

In [6]:
# Check if all variables exist
DATA_DIR = Path(DATA_DIR).resolve()
AN_missing_gene_dict = Path(AN_missing_gene_dict).resolve()

if not DATA_DIR.exists() and not DATA_DIR.is_dir():
    raise FileNotFoundError(f"Data directory {DATA_DIR} does not exist or is not a directory.")

if not AN_missing_gene_dict.exists() and not AN_missing_gene_dict.is_file():
    raise FileNotFoundError(f"AN missing gene dictionary {AN_missing_gene_dict} does not exist or is not a file.")

if not most_recent_gdt_file:
    if nth_iteration > 2:
        raise FileNotFoundError(f"Most recent GDT file {most_recent_gdt_file} does not exist or is not a file.")
    else:
        print(f"If you set up a stripped GDT file, please set the path to it in the most_recent_gdt_file variable.")
        print(f"Otherwise, ignore this message.")
else:
    most_recent_gdt_file = Path(most_recent_gdt_file).resolve()
    if not most_recent_gdt_file.exists() and not most_recent_gdt_file.is_file():
        raise FileNotFoundError(f"Most recent GDT file {most_recent_gdt_file} does not exist or is not a file.")

If you set up a stripped GDT file, please set the path to it in the most_recent_gdt_file variable.
Otherwise, ignore this message.


In [7]:
_, logger = gdt.logger_setup.logger_creater(log_file="test.log", console_level="DEBUG", file_level="TRACE")
logger.debug("Running from notebook AN_missing_gene_dict")

2025-05-07 18:29:42,925 - DEBUG - Logger setup complete. Logging to /home/brenodupin/matheus/gdt/notebooks/test.log
2025-05-07 18:29:42,926 - DEBUG - Running from notebook AN_missing_gene_dict


In [ ]:
with open(AN_missing_gene_dict, "r") as f:
    ANs = [line.strip() for line in f.readlines() if line.strip()]
print(f"len(ANs): {len(ANs)}")

In [ ]:
def data_process(df_missing, AN, gene_dict, temp_gene_dict, organelle_type, temp_count, logger, use_symbol=False, temp_name='temp_desc'):
    """
    Process the data in the dataframe and update the gene_dict and corresponding temp_gene_dict.
    Args:
        df_missing (pd.DataFrame): DataFrame containing the missing genes.
        AN (str): The accession number.
        gene_dict (dict): Dictionary containing gene information.
        temp_gene_dict (dict): Temporary dictionary for gene information.
        organelle_type (str): Type of organelle. Should be "MT" or "PT".
        temp_count (int): Counter for temporary labels.
        logger: Logger object for logging messages.
        use_symbol (bool): Flag to indicate whether to use gene symbol or description. Default is False, which means use description.
        temp_name (str): Name for the temporary dictionary. Default is 'temp_desc'.
    Returns:
        tuple: Updated gene_dict, temp_gene_dict, and temp_count.
    """
    for row in df_missing.itertuples():
        check_var = row.gene_symbol if use_symbol else row.desc

        logger.trace(f'gene_id: {row.gene_id} | dbxref: {row.dbxref} | s: {row.start} | att: {row.attributes}')
        logger.trace(f'\tname: {row.other_aliases} | desc: {row.desc} | gene_symbol: {row.gene_symbol}')
        logger.trace(f'\tcheck_var: {check_var} | use_symbol: {use_symbol}')        
        
        if check_var in gene_dict:
            gene_label = gene_dict[check_var].label
            logger.debug(f'[1st T]Label in gene_dict, L: |{gene_label}| adding: {row.gene_id} #gn {AN}:{row.dbxref} #c ncbi_desc: {check_var}')
            gene_dict[row.gene_id] = gdt.gene_dict.GeneDbxref(
                    label=gene_label,
                    an_source=AN,
                    dbxref=row.dbxref,
                    c=f'ncbi_desc: {check_var}')
        
        else:
            logger.trace(f'[1st F]Label not found gene_dict | checking {temp_name} | Label: {check_var}')

            if check_var in temp_gene_dict:
                gene_label = temp_gene_dict[check_var].label
                logger.debug(f'[2nd T]Label in {temp_name}, L: |{gene_label}| adding: {row.gene_id} #gn {AN}:{row.dbxref} #c ncbi_desc: {check_var}')
                temp_gene_dict[row.gene_id] = gdt.gene_dict.GeneDbxref(
                    label=gene_label,
                    an_source=AN,
                    dbxref=row.dbxref,
                    c=f'ncbi_desc: {check_var}')
            else:
                
                temp_count += 1
                label = f'{organelle_type}-TEMP-{temp_count}'
                logger.debug(f'[2nd F]Label not in {temp_name}, new label |{label}| adding: {row.gene_id} #gn {AN}:{row.dbxref} #c ncbi_desc: {check_var}')
                temp_gene_dict[check_var] = gdt.gene_dict.GeneDescription(
                    label=label,
                    source="NCBI",
                    c=None)

                temp_gene_dict[row.gene_id] = gdt.gene_dict.GeneDbxref(
                    label=label,
                    an_source=AN,
                    dbxref=row.dbxref,
                    c=f'ncbi_desc: {check_var}')
    
    return gene_dict, temp_gene_dict, temp_count

### TEMP using NCBI description

In [ ]:
# Load the GDT file
if most_recent_gdt_file:
    gene_dict = gdt.gene_dict.create_gene_dict(most_recent_gdt_file, max_an_sources=0)
else:
    gene_dict = {}

temp_gene_dict = {}

In [ ]:
temp_gene_dict = {}
temp_count = 0
errors = []
logger.info(' ---- [Starting TEMP process] ----')
for i, AN in enumerate(ANs):
    logger.debug(f'-- [Processing: {AN}] --')
    
    an_path = DATA_DIR / f'{AN}{gff_suffix}'
    if not an_path.exists():
        logger.error(f'Error: {AN} does not exist (an_path: {an_path})')
        errors.append((AN, 'File not found'))
        continue
    
    df = gdt.gff3_utils.load_gff3(an_path, query_string='type == "gene"')
    df = gdt.gff3_utils.filter_orfs(df) if remove_orfs else df

    # getting the gene_id and if it is in the gene_dict
    df['gene_id'] = df['attributes'].str.split(';').str[0].str.replace('ID=', '', regex=False)
    df['in_gene_dict'] = df['gene_id'].isin(gene_dict)
    df_missing = df[~df['in_gene_dict']].copy()

    # two step method to extract dbxref, first try to get the full dbxref,
    # if not all genes are numeric and not NaN, fallback to GeneID,
    # check again if all genes are numeric and not NaN.
    # if not, raise an error
    df_missing['dbxref'] = df_missing['attributes'].str.extract(r'Dbxref=GeneID:([^;]*)(?:;|$)')
    if df_missing['dbxref'].isna().any() or not df_missing['dbxref'].str.isnumeric().all():
        logger.warning(f'Error in {AN} - dbxref is not numeric or contains NaN')
        logger.debug('\ttrying only "GeneID:"')
        
        df_missing['dbxref'] = df_missing['attributes'].str.extract(r'GeneID:([^;]*)(?:;|$)')
        if not df_missing['dbxref'].str.isnumeric().all() or df_missing['dbxref'].isna().any():
            logger.error(f'\tError in {AN} - dbxref is not numeric or contains NaN x2')
            errors.append((AN, 'NaN or not numeric'))
            continue
        else:
            logger.debug('\tSuccess! - continuing')

    # search NCBI
    with Entrez.esummary(db="gene", id=",".join(df_missing['dbxref'])) as search_handle:
        try:
            search_results = Entrez.read(search_handle)['DocumentSummarySet']['DocumentSummary'] # type: ignore
        except (RuntimeError, KeyError, Exception) as ex:
            logger.error(f'{ex} in Entrez.read for {AN}')
            errors.append((AN, 'Entrez.read'))
            continue
    
    mr_check = len(df_missing) == len(search_results)
    logger.trace(f"\tm: {len(df_missing)} | r: {len(search_results)} | m/r check: {mr_check}") # type: ignore
    
    # merge with search_results
    temp_df = pd.DataFrame([{
        'dbxref': x.attributes['uid'],
        'other_aliases': x.get('OtherAliases', 'no_other_aliases'),
        'desc': x.get('Description', 'no_description'),
        'gene_symbol': x.get('Name', 'no_gene_symbol')
        } for x in search_results])
    
    df_missing = df_missing.merge(temp_df, on='dbxref', how='left', copy=False)

    # check if df_missing len is equal to search_results, and equal to the original df
    if (len(df_missing) != len(temp_df)) or (len(df_missing) != len(df[~df['in_gene_dict']])):
        logger.warning(f'{AN} m/r check: {mr_check} | df_missing len {len(df_missing)} | temp_df len {len(temp_df)} | df len {len(df[~df["in_gene_dict"]])}')
        logger.warning(f'This is not expected, but can be caused by fragmented genes that have the same dbxref/gene_id. Please check the log file for more details in TRACE level.')
    
    # process the data
    gene_dict, temp_gene_dict, temp_count = data_process(df_missing, AN, gene_dict,
                                                         temp_gene_dict, organelle_type,
                                                         temp_count, logger)
    

logger.info(f' ---- [Finished] ----')
if errors:
    logger.warning(f'Errors: {len(errors)}')
    for an, msg in errors:
        logger.warning(f'{an} - {msg}')

In [12]:
if temp_gene_dict:
    temp_gene_dict['info'] = gdt.gene_dict.get_gene_dict_info(temp_gene_dict)
    temp_gene_dict['header'] = ['version 0.0.2', f'TEMP - {nth_iteration}', 'Automagically generated by gdt.py']
    gdt.gene_dict.write_gdt_file(temp_gene_dict, f'TEMP_{nth_iteration:02}.gdt', overwrite=True)

In [ ]:
if gene_dict:
    gene_dict['info'] = gdt.gene_dict.get_gene_dict_info(gene_dict)
    gene_dict['header'] = ['version 0.0.2', f'GDT - {nth_iteration}', 'Automagically generated by gdt.py', f'Data added from TEMP {nth_iteration:02}']
    gdt.gene_dict.write_gdt_file(gene_dict, f'GDT_pilot_{nth_iteration:02}.gdt', overwrite=True)

### TEMP using NCBI Symbol

In [ ]:
# Load the GDT file
if most_recent_gdt_file:
    gene_dict = gdt.gene_dict.create_gene_dict(most_recent_gdt_file, max_an_sources=0)
else:
    gene_dict = {}

temp_symbol_gene_dict = {}

In [ ]:
temp_symbol_gene_dict = {}
temp_count = 0
errors = []
logger.info(' ---- [Starting TEMP process] ----')
for i, AN in enumerate(ANs):
    logger.debug(f'-- [Processing: {AN}] --')
    
    an_path = DATA_DIR / f'{AN}{gff_suffix}'
    if not an_path.exists():
        logger.error(f'Error: {AN} does not exist (an_path: {an_path})')
        errors.append((AN, 'File not found'))
        continue
    
    df = gdt.gff3_utils.load_gff3(an_path, query_string='type == "gene"')
    df = gdt.gff3_utils.filter_orfs(df) if remove_orfs else df

    # getting the gene_id and if it is in the gene_dict
    df['gene_id'] = df['attributes'].str.split(';').str[0].str.replace('ID=', '', regex=False)
    df['in_gene_dict'] = df['gene_id'].isin(gene_dict)
    df_missing = df[~df['in_gene_dict']].copy()

    # two step method to extract dbxref, first try to get the full dbxref,
    # if not all genes are numeric and not NaN, fallback to GeneID,
    # check again if all genes are numeric and not NaN.
    # if not, raise an error
    df_missing['dbxref'] = df_missing['attributes'].str.extract(r'Dbxref=GeneID:([^;]*)(?:;|$)')
    if df_missing['dbxref'].isna().any() or not df_missing['dbxref'].str.isnumeric().all():
        logger.warning(f'Error in {AN} - dbxref is not numeric or contains NaN')
        logger.debug('\ttrying only "GeneID:"')
        
        df_missing['dbxref'] = df_missing['attributes'].str.extract(r'GeneID:([^;]*)(?:;|$)')
        if not df_missing['dbxref'].str.isnumeric().all() or df_missing['dbxref'].isna().any():
            logger.error(f'\tError in {AN} - dbxref is not numeric or contains NaN x2')
            errors.append((AN, 'NaN or not numeric'))
            continue
        else:
            logger.debug('\tSuccess! - continuing')

    # search NCBI
    with Entrez.esummary(db="gene", id=",".join(df_missing['dbxref'])) as search_handle:
        try:
            search_results = Entrez.read(search_handle)['DocumentSummarySet']['DocumentSummary'] # type: ignore
        except (RuntimeError, KeyError, Exception) as ex:
            logger.error(f'{ex} in Entrez.read for {AN}')
            errors.append((AN, 'Entrez.read'))
            continue
    
    mr_check = len(df_missing) == len(search_results)
    logger.trace(f"\tm: {len(df_missing)} | r: {len(search_results)} | m/r check: {mr_check}") # type: ignore
    
    # merge with search_results
    temp_df = pd.DataFrame([{
        'dbxref': x.attributes['uid'],
        'other_aliases': x.get('OtherAliases', 'no_other_aliases'),
        'desc': x.get('Description', 'no_description'),
        'gene_symbol': x.get('Name', 'no_gene_symbol')
        } for x in search_results])
    
    df_missing = df_missing.merge(temp_df, on='dbxref', how='left', copy=False)

    # check if df_missing len is equal to search_results, and equal to the original df
    if (len(df_missing) != len(temp_df)) or (len(df_missing) != len(df[~df['in_gene_dict']])):
        logger.warning(f'{AN} m/r check: {mr_check} | df_missing len {len(df_missing)} | temp_df len {len(temp_df)} | df len {len(df[~df["in_gene_dict"]])}')
        logger.warning(f'This is not expected, but can be caused by fragmented genes that have the same dbxref/gene_id. Please check the log file for more details in TRACE level.')
    
    # process the data
    gene_dict, temp_symbol_gene_dict, temp_count = data_process(df_missing, AN, gene_dict,
                                                         temp_symbol_gene_dict, organelle_type,
                                                         temp_count, logger, temp_name='temp_symbol', use_symbol=True)
    

logger.info(f' ---- [Finished] ----')
if errors:
    logger.warning(f'Errors: {len(errors)}')
    for an, msg in errors:
        logger.warning(f'{an} - {msg}')

In [ ]:
if temp_symbol_gene_dict:
    temp_symbol_gene_dict['info'] = gdt.gene_dict.get_gene_dict_info(temp_symbol_gene_dict)
    temp_symbol_gene_dict['header'] = ['version 0.0.2', f'TEMP Symbol - {nth_iteration}', 'Automagically generated by gdt.py']
    gdt.gene_dict.write_gdt_file(temp_symbol_gene_dict, f'TEMP_{nth_iteration:02}.gdt', overwrite=True)

In [ ]:
if gene_dict:
    gene_dict['info'] = gdt.gene_dict.get_gene_dict_info(gene_dict)
    gene_dict['header'] = ['version 0.0.2', f'GDT - {nth_iteration}', 'Automagically generated by gdt.py', f'Data added from TEMP Symbol {nth_iteration:02}']
    gdt.gene_dict.write_gdt_file(gene_dict, f'GDT_pilot_{nth_iteration:02}.gdt', overwrite=True)

### 'Removing' genes

In [31]:
remove_string = 'discard-'
genes_to_remove = "gene_to_remove"

gene-J2C28_mgp19 #dx NC_053825.1:63373456 #c ncbi_desc: hypothetical protein
gene-J2C28_mgp16 #dx NC_053825.1:63373460 #c ncbi_desc: hypothetical protein

In [32]:
stuff_to_remove = {}
with open(genes_to_remove, "r") as f:
    for line in f:
        if not line.strip():
            continue
        
        an, dbxref = line.split("#c", 1)[0].split("#dx", 1)[1].strip().split(":")
        
        if not an or not dbxref:
            raise ValueError(f"Error: {line} - AN and dbxref are empty after split, why?")
        
        if an not in stuff_to_remove:
            stuff_to_remove[an] = [dbxref]
        else:
            stuff_to_remove[an].append(dbxref)

In [33]:
stuff_to_remove

{'NC_053825.1': ['63373456', '63373460']}

In [ ]:
#TODO check better the dbxref extract for the gff3, since we are getting the whole gff3, cds and all
for an in stuff_to_remove.keys():
    an_path = DATA_DIR / f'{an}{gff_suffix}'

    df = gdt.gff3_utils.load_gff3(an_path, usecols=gdt.gff3_utils.GFF3_COLUMNS)
    
    df['dbxref'] = df['attributes'].str.extract(r'Dbxref=GeneID:([^;]*)(?:;|$)')
    if df.loc[1:, 'dbxref'].isna().any() or not df.loc[1:, 'dbxref'].str.isnumeric().all():
        print(f'Error in {an} - dbxref is not numeric or contains NaN')
        print('\ttrying only "GeneID:"')
        
        df['dbxref'] = df['attributes'].str.extract(r'GeneID:([^;]*)(?:;|$)')
        if df.loc[1:, 'dbxref'].isna().any() or not df.loc[1:, 'dbxref'].str.isnumeric().all():
            raise ValueError(f'\tError in {an} - dbxref is not numeric or contains NaN x2')
        else:
            print('Success! - continuing')

    mask = df['dbxref'].isin(stuff_to_remove[an])
    df.loc[mask, 'type'] = remove_string + df.loc[mask, 'type']
    
    with open(an_path, 'r+') as f:
        headers = ''
        for line in f:
            if not line.startswith('#'):
                break
            headers += line
        f.seek(0)
        f.write(headers)
        df.to_csv(f, sep='\t', index=False, header=False)
        f.write('\n')
    print(f'Updated {an} - {len(df.loc[mask])} genes marked for removal')
    
    print()

    #show the whole df

            

Error in NC_053825.1 - dbxref is not numeric or contains NaN
	trying only "GeneID:"
Success! - continuing
Updated NC_053825.1 - 4 genes marked for removal



In [35]:
df

,seqid,source,type,start,end,score,strand,phase,attributes,dbxref
0,NC_053825.1,RefSeq,region,1,59014,.,+,.,ID=NC_053825.1:1..59014;Dbxref=taxon:1290591;I...,NaN
1,NC_053825.1,RefSeq,gene,1,7676,.,+,.,ID=gene-J2C28_mgr02;Dbxref=GeneID:63373489;Nam...,63373489
2,NC_053825.1,RefSeq,rRNA,1,7676,.,+,.,ID=rna-J2C28_mgr02;Parent=gene-J2C28_mgr02;Dbx...,63373489
3,NC_053825.1,RefSeq,exon,1,627,.,+,.,ID=exon-J2C28_mgr02-1;Parent=rna-J2C28_mgr02;D...,63373489
4,NC_053825.1,RefSeq,gene,1055,1276,.,+,.,ID=gene-J2C28_mgp30;Dbxref=GeneID:63373439;Nam...,63373439
...,...,...,...,...,...,...,...,...,...,...
151,NC_053825.1,RefSeq,tRNA,58840,58911,.,+,.,ID=rna-J2C28_mgt22;Parent=gene-J2C28_mgt22;Dbx...,63373487
152,NC_053825.1,RefSeq,exon,58840,58911,.,+,.,ID=exon-J2C28_mgt22-1;Parent=rna-J2C28_mgt22;D...,63373487
153,NC_053825.1,RefSeq,gene,58923,58994,.,+,.,ID=gene-J2C28_mgt23;Dbxref=GeneID:63373488;Nam...,63373488
154,NC_053825.1,RefSeq,tRNA,58923,58994,.,+,.,ID=rna-J2C28_mgt23;Parent=gene-J2C28_mgt23;Dbx...,63373488


#